- references
  - https://www.uio.no/studier/emner/matnat/ifi/INF3380/v11/undervisningsmateriale/inf3380-week06.pdf
  - https://indico.desy.de/event/12535/contributions/9450/attachments/6466/7452/CL5_MPI_Lecture.pdf

## overall

> 都算是并行计算库（library）；

- NCCL: NVIDIA Collective Communication Library (NCCL)
    - PCIe、Nvlink、InfiniBand 上可以实现较高的通信速度。
- MPI: Message Passing Interface
- gloo: 单机多进程的情况 on CPU

## MPI

## NCCL

In [1]:
!export NCCL_DEBUG=INFO
!export NCCL_SOCKET_IFNAME=eno2
# export NCCL_IB_DISABLE=1

In [2]:
import torch
import torch.distributed as dist
import os
from datetime import datetime, timedelta

In [3]:
def run(rank, size):
    """ Distributed function to be implemented later. """
    pass

def init_process(rank, size, fn, backend='nccl'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = 'localhost'
#     os.environ['MASTER_PORT'] = '29501'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group(backend, rank=rank, world_size=size, 
                            timeout=timedelta(days=2)
                           )
    fn(rank, size)    

In [4]:
size = 2  # Number of processes
processes = []
for rank in range(size):
    p = torch.multiprocessing.Process(target=init_process, args=(rank, size, run))
    p.start()
    processes.append(p)

for p in processes:
    p.join()

Process Process-2:
Traceback (most recent call last):
  File "/home/whaow/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/whaow/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_50556/2435254016.py", line 10, in init_process
    dist.init_process_group(backend, rank=rank, world_size=size,
  File "/home/whaow/anaconda3/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py", line 932, in init_process_group
    _store_based_barrier(rank, store, timeout)
  File "/home/whaow/anaconda3/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py", line 456, in _store_based_barrier
    worker_count = store.add(store_key, 0)
RuntimeError: Broken pipe


## Gloo